In [1]:
from dotenv import load_dotenv

load_dotenv()

from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_text_splitters.base import Language
from langchain_community.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.chat_models import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import (
    HumanMessage,AIMessage
)

import gradio as gr
import json
import openai

c:\Users\oksk1\venv\lc_rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125")

# llm
# llm=ChatOpenAI(
#     streaming=True,
#     callbacks=[StreamingStdOutCallbackHandler()],
#     model="gpt-3.5-turbo-0125",
#     temperature=0,
# )

c:\Users\oksk1\venv\lc_rag\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [3]:
# 해당 부분은 이미 임베딩된 데이터를 로드하는 부분이다.

vectorstore = Chroma(persist_directory='../data/chroma_db', embedding_function=OpenAIEmbeddings())

c:\Users\oksk1\venv\lc_rag\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [4]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})
#prompt = hub.pull("rlm/rag-prompt")
prompt = hub.pull("hwchase17/openai-tools-agent")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [5]:
qa_interface = RetrievalQA.from_chain_type(llm=llm,
                                           chain_type="stuff",
                                           retriever=retriever,
                                           chain_type_kwargs={
                                            "verbose": True,
                                           },
                                           return_source_documents=True)

In [8]:
with gr.Blocks() as rag_tester:
    gr.HTML("법률 사례기반 챗봇")

    with gr.Row():
        with gr.Column(scale=1):
            chatbot = gr.Chatbot()
            msg = gr.Textbox(placeholder="법률 관련 질문을 입력하세요.")
        with gr.Column(scale=1):
            gr.HTML("검색 문서")
            source = gr.Textbox()

    def userHandler(user_message, history):
        #print("@@ history:", history)
        return "", history + [[user_message, None]]

    def botHandler(history):
        qa_result = qa_interface(history[-1][0])

        history[-1][1] = qa_result["result"]
        source_text = ""
        for ix, doc in enumerate(qa_result['source_documents']):
            source_text += f"## 검색문서 {ix+1}\n```\n{doc.page_content}\n```\n\n"

        # for i in range(len(history)):
        #     print(f"### {i}번째\nQuestion: {history[i][0]}\nAnswer: {history[i][1]}")
        print("## history:", history)

        return history, source_text

    msg.submit(userHandler, [msg, chatbot], [msg, chatbot], queue=False).then(
        botHandler, chatbot, [chatbot, source]
    )

rag_tester.launch(debug=True, share=True)

@@ msg: <gradio.components.textbox.Textbox object at 0x000001BB7A67FB10>
Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
"안녕하세요 저는 휴대폰 매장 대표 입니다, 직원이 돈이 없엇어 퀵비를 안줫다구 합ㅈ니다. 제가 그 거땜에 퀵기사님 상대로 저 를 고소를 햇다구합니다.저는 드릴러고 햇는데 자꾸 안받는다구 햇어고 그래서 매장에서 도 매번 오서는데 제가 드릴러고 햇어는데 자꾸 안받는다 그래서 매장 근처 돌면서 동영상 도찍으면서 돈을자체 안받는다구 하섯고 그래서 저 상대러 고소 햇다구 합니다, 그래서 오늘 경찰 출석인데 제가 늦쳣습니다 낼 경찰 출석인데 변호사님들이랑 같이 동해 하고 싶은데 비용이 문제라서 ㅠㅠㅠ 그래서 이렇게 햇습니다 낼 변호사님이랑 동행 할수잇으가요",,불구속 수사중인 경우 국선변호사 제도가 없으며 변호사 선임을 원하면 개인적으로 선임하여야 합니다.
Human: 안녕하세요

> Finished chain.

> Finished chain.
### 0번째
Question: 안녕하세요
Answer: 안녕하세요, 어떻게 도와드릴까요?


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know

In [25]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain.prompts import MessagesPlaceholder

In [7]:
from langchain.callbacks.base import BaseCallbackHandler

class MyCustomHandler(BaseCallbackHandler):

    def __init__(self, domain=None, details=None):
        self.domain = domain
        self.details = details

    def on_tool_start(self, serialized: dict[str, any], **kwargs: any) -> any:
        """Run when tool starts running."""
        print(f"on_tool_start (A tool is starting!!)")
        print("#", serialized)
        print("#", kwargs)
        self.domain = serialized

    def on_tool_start(
        self, serialized: dict[str, any], input_str: str, **kwargs: any
    ) -> any:
        """Run when tool starts running."""
        print(f"on_tool_start (I'm starting!!)")
        print("#", serialized) # 이 값으로 어떤것으로 분류 되었는지 알 수 있다.
        print("#", input_str)
        print("#", kwargs)
        self.domain = serialized

    def on_tool_end(self, output: str, **kwargs: any) -> any:
        """Run when tool ends running."""
        print(f"on_tool_end (I'm ending!!)")
        print("#", output) # 이 값으로 링크만 따로 분류 할 수 있다.
        print("#", kwargs)
        self.details = output

In [29]:
# llm
llm=ChatOpenAI(
    model="gpt-3.5-turbo-0125",
    temperature=0
)

# prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            당신은 법률기반 질의응답에 기반해 사용자의 질문에 답변하는 챗봇입니다.
            반드시 전달받은 context에 기반해 답변해야 하며,
            전달받은 context로 답변할 수 없을 시 답변을 절대 하면 안 됩니다.
            Lets' think step by step.
            """
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad") # 메세지 리스트를 전달하는 공간
    ]
)

retriever_tool = create_retriever_tool(
    retriever,
    "law_qa_search",
    "법률에 대한 질문이라면 반드시 이 tool을 사용하세요.",
)


# search
search = TavilySearchResults(max_results=1)

# tools
# tools = [retriever_tool]
tools = [retriever_tool, search]


# # 언어체인
# combine_docs_chain = create_stuff_documents_chain(llm, prompt)
# # 검색체인
# retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


customHandler = MyCustomHandler()

result1 = agent_executor.invoke({"input": "근로기준법상 근로자에는 정규직만 들어가는 건가요?"})
result2 = agent_executor.invoke({
    "input": "연예인 뉴진스에 대해 설명해주세요."
    }, 
    {
        "callbacks": [customHandler]
    })

print(result2)
print("domain:", customHandler.domain)
print("details:", customHandler.details)



> Entering new AgentExecutor chain...

Invoking: `law_qa_search` with `{'query': '근로기준법 근로자 정의'}`


근무하는 한국인 노동자에게는 「근로기준법」및 「산업재해보상보험법」이 적용됩니다(산업재해보상보험법 제122조).다만, 상시 5인 이상의 근로자를 사용하는 사업 또는 사업장이라도 다음의 경우에는 「근로기준법」이 적용되지 않습니다.첫째, 동거의 친족만을 사용하는 사업은 종속적 근로관계를 인정할 수 없으니 친족상호간에 누가 사용자이고 노동자인지를 형식적으로 구별하기 곤란하기 때문에 법적용에서 제외하고 있습니다. 그러나 「민법」제777조에 의한 법률상의 친족범위(8촌 이내의 혈족, 4촌 이내의 인척, 배우자)밖에 있는 친족을 근로자로 사용하는 경우에는 「근로기준법」이 적용됩니다.둘째, 가사사용인은 제외됩니다. 가사사용인이란 일반가정의 가정부, 파출부 등 가사에 종사하는 자를 말합니다. 가사사용인과의 관계는 주로 개인의 사생활과 관련되어 있고 근무시간이나 임금규제를 통하여 국가의 감독행정이 미치기 어렵기 때문입니다.셋째, 특별법에 의한 적용의 예외로서 선원의 경우는 해상노동의 특이성을 고려하여 선원법이 적용되며, 국가공무원 및 지방공무원은 특별규정(예：공무원연금법, 공무원보수규정 등)이 있는 경우에는 「근로기준법」의 적용이 제한되며, 사립학교교직원은 「사립학교법」에 특별히 「근로기준법」의 적용을 배제한다는 규정이 없는 한 「근로기준법」이 적용됩니다.또한 판례는 “근로기준법 제10조(현행 근로기준법 제11조) 본문에서 ‘상시 5인 이상의 근로자를 사용하는 사업 또는 사업장’이라고 함은 근로자 수가 때때로 5인 미만이 되는 경우가 있어도 상태적으로 5인 이상이 되는 경우에는 이에 해당하고, 나아가 당해 사업장에 계속 근무하는 근로자뿐 아니라 그때 그때의 필요에 의하여 사용하는 일용근로자도 포함하는 것이다.”라고 하였습니다(대법원 1997. 11. 28. 선고 97다28971 판결, 2000. 3

# Agents 접목

In [27]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [30]:
# 1. Load, chunk and index the contents of the blog to create a retriever.

vectorstore = Chroma(persist_directory='../data/chroma_db', embedding_function=OpenAIEmbeddings())

In [31]:
retriever = vectorstore.as_retriever()

In [35]:
from langgraph.prebuilt import chat_agent_executor
from langchain.tools.retriever import create_retriever_tool

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
retriever = vectorstore.as_retriever()

tool = create_retriever_tool(
    retriever,
    "law_qna_retriever",
    "Searches and returns exceprts from the Autonomous Agents Q&A."
)
tools = [tool]

config = {"configurable": {"thread_id": "abc123"}}
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

agent_executor = chat_agent_executor.create_tool_calling_executor(
    llm, tools, checkpointer=memory
)



customHandler = MyCustomHandler()

# result1 = agent_executor.invoke({"input": "근로기준법상 근로자에는 정규직만 들어가는 건가요?"})
# # result2 = agent_executor.invoke({
# #     "input": "연예인 뉴진스에 대해 설명해주세요."
# #     }, 
# #     {
# #         "callbacks": [customHandler]
# #     })
# result2 = agent_executor.invoke({"input": '최저임금은?'})

# print(result2)
# print("domain:", customHandler.domain)
# print("details:", customHandler.details)


ValueError: Checkpointer requires one or more of the following 'configurable' keys: ['thread_id', 'thread_ts']

In [36]:
query = "근로기준법상 근로자에는 정규직만 들어가는 건가요?"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Fk4Wcx3mw90oSWK9byI3gWx4', 'function': {'arguments': '{"query":"근로기준법 근로자 정규직"}', 'name': 'law_qna_retriever'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 97, 'total_tokens': 133}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-87fdba32-cc21-4a67-a89c-ff66db315efc-0', tool_calls=[{'name': 'law_qna_retriever', 'args': {'query': '근로기준법 근로자 정규직'}, 'id': 'call_Fk4Wcx3mw90oSWK9byI3gWx4'}])]}}
----
{'tools': {'messages': [ToolMessage(content='평소 4명이서 일하는 사업장에서 예약이 밀리거나 할 때 일용직을 고용하는 경우도 상시 근로자에 해당되나요?,,"대법원 2008. 3. 27. 선고 2008도364 판결에 따르면 ‘상시 5인 이상의 근로자를 사용하는 사업 또는 사업장’이라 함은 ‘상시 근무하는 근로자의 수가 5인 이상인 사업 또는 사업장’이 아니라 ‘사용하는 근로자의 수가 상시 5인 이상인 사업 또는 사업장’을 뜻하는 것이고, 이 경우 상시라 함은 상태라고 하는 의미로서 근로자의 수가 때때로 5인 미만이 되는 경우가 있어도 사회통념에 의하여 객관적으로 판단하여 상태적으로 5인 이상이 되는 경우에는 이에 해당한다 할 것이고, 여

In [37]:
query = "최저임금은"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8kaIo3vZcqfFJq2eCOaFLIlH', 'function': {'arguments': '{"query":"최저임금"}', 'name': 'law_qna_retriever'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 3646, 'total_tokens': 3671}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6a902038-4bff-455e-9ca8-6f856be40980-0', tool_calls=[{'name': 'law_qna_retriever', 'args': {'query': '최저임금'}, 'id': 'call_8kaIo3vZcqfFJq2eCOaFLIlH'}])]}}
----
{'tools': {'messages': [ToolMessage(content='최저임금안을 심의하며, 심의위원회로부터 최저임금안을 제출받은 때에는 지체없이 사업의 종류별 최저임금안 및 적용사업의 범위를 고시하여야 하며, 매년 8월 5일까지 최저임금을 결정하여야 합니다(같은 법 제4조, 제5조, 제8조, 제9조, 같은 법 시행령 제7조).참고로 고용노동부에서 고시한 2016년 1월 1일부터 2016년 12월 31일 까지 적용되는 최저임금액을 보면 시간급은 6,030원으로 정하고 있습니다."\n\n"최저임금제도란 무엇이며, 최저임금액은 어떻게 결정되는지요?",,"최저임금제도란 국가가 임금액의 최저한도를 정하여 사용자에게 이를 준수하도록 강제하는 제도를 말합니다. 그러므로 최저임금액이 결정·고시되면 

In [38]:
query = "뉴진스 사태는"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pjrdjMeZ1nBRZSwZO4AWkeTS', 'function': {'arguments': '{"query":"뉴진스 사태"}', 'name': 'law_qna_retriever'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 5884, 'total_tokens': 5909}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-17c9c242-10d7-474b-9a60-5ad78d36b143-0', tool_calls=[{'name': 'law_qna_retriever', 'args': {'query': '뉴진스 사태'}, 'id': 'call_pjrdjMeZ1nBRZSwZO4AWkeTS'}])]}}
----
{'tools': {'messages': [ToolMessage(content='"국내 주식형 펀드에 투자했다가, 유럽재정 위기악화로 인하여 큰 손실을 입었습니다. 손해를 배상받고 싶은데요, 어떻게 해야 하나요?",,"펀드와 관련하여 분쟁이 있는 투자자는 금융감독원장에게 분쟁의 조정을 신청할 수 있으며, 한국소비자원의 소비자상담센터에 민원을 제기하거나, 한국거래소 또는 금융투자협회의 분쟁조정위원회의 자율조정제도를 통해 분쟁을 해결할 수 있습니다. 금융감독원이나 한국소비자원 등을 통한 조정절차에서 해결이 되지 않은 경우에는 법원의 민사소송이나 증권관련집단소송을 통해 권리를 구제받을 수도 있습니다."\n영유아 건강검진이란 무엇이고 언제 실시하나요?,,"""영유아 건강검진""이란 국민건강보험공단